Write a function that takes porker hands and returns the best hand.

Ranking rules: https://en.wikipedia.org/wiki/List_of_poker_hands#Hand-ranking_categories

In [1]:
# from enum import Enum, auto

# class HandName(Enum):
#     five_of_kind = auto()
#     straight_flush = auto()
#     four_of_kind = auto()
#     full_house = auto()
#     flush = auto()
#     straight = auto()
#     three_of_kind = auto()
#     two_pair = auto()
#     one_pair = auto()
#     high_card = auto()

**Representing cards**

It's better to do it by using a list of tuples `[("J", "S"), ("A", "D")]` or a list `["JS", "AD"...]`. If we have *wildcards* or duplicate decks, a set will not work.

I think the best one would be like:

`[("11", "S"), ("1", "D"), ...]`

In [2]:
sf = "6C 7C 8C 9C TC".split() # Straight Flush
fk = "9D 9H 9S 9C 7D".split() # Four of a Kind
fh = "TD TC TH 7C 7D".split() # Full House
tp = "5S 5D 9H 9C 6S".split() # Two pairs

In [3]:
def poker(hands):
    "Return the best hand: poker([hand,...]) => hand"
    return max(hands, key=hand_rank)


def hand_rank(hand):
    """Return a value indicating the ranking of a hand."""
    ranks = card_ranks(hand)
    if straight(ranks) and flush(hand):
        return (8, max(ranks))
    elif kind(4, ranks):
        return (7, kind(4, ranks), kind(1, ranks))
    elif kind(3, ranks) and kind(2, ranks):
        return (6, kind(3, ranks), kind(2, ranks))
    elif flush(hand):
        return (5, ranks)
    elif straight(ranks):
        return (4, max(ranks))
    elif kind(3, ranks):
        return (3, kind(3, ranks), ranks)
    elif two_pair(ranks):
        return (2, two_pair(ranks), ranks)
    elif kind(2, ranks):
        return (1, kind(2, ranks), ranks)
    else:
        return (0, ranks)

In [4]:
def card_ranks(hand):
    "Return a list of the ranks, sorted with higher first."
    ranks = ["--23456789TJQKA".index(r) for r, s in hand]
    ranks.sort(reverse=True)
    return ranks

In [5]:
def flush(hand):
    "Return True if all the cards have the same suit."
    suits = [e[-1] for e in hand]
    for s in suits[1:]:
        if suits[0] != s:
            return False
    return True

Peter

In [6]:
def flush(hand):
    "Return True if all the cards have the same suit."
    suits = [s for r,s in hand]
    return len(set(suits)) == 1

In [7]:
flush("6C 7C 8C 9C TC".split())

True

In [8]:
def straight(ranks):
    "Return True if the ordered ranks form a 5-card straight."
    ranks = sorted(ranks, reverse=True)
    return all([ranks[i] == ranks[i + 1] + 1 for i in range(len(ranks) - 1)])
    # Your code here.

Peter implementation:

In [9]:
def straight(ranks):
    "Return True if the ordered ranks form a 5-card straight."
    return (max(ranks) - min(ranks) == 4) and len(set(ranks)) == 5

In [10]:
assert straight([9, 8, 7, 6, 5]) == True

In [11]:
assert straight([9, 8, 8, 6, 5]) == False

In [12]:
from collections import Counter


def kind(n, ranks):
    """Return the first rank that this hand has exactly n of.
    Return None if there is no n-of-a-kind in the hand."""
    
    for k, v in Counter(ranks).items():

        if v == n:
            return k
        
    return None

Peter

In [13]:
def kind(n, ranks):
    """Return the first rank that this hand has exactly n of.
    Return None if there is no n-of-a-kind in the hand."""
    
    for r in ranks:
        if ranks.count(r) == n:
            return r
    return None


In [14]:
fkranks = card_ranks(fk)
tpranks = card_ranks(tp)
assert kind(4, fkranks) == 9
assert kind(3, fkranks) == None
assert kind(2, fkranks) == None
assert kind(1, fkranks) == 7

In [15]:
def two_pair(ranks):
    """If there are two pair, return the two ranks as a
    tuple: (highest, lowest); otherwise return None."""

    pairs = set([e for e in ranks if ranks.count(e) == 2])

    if len(pairs) != 2:
        return None

    return tuple(sorted(pairs, reverse=True))

In [16]:
assert two_pair([1, 2, 1, 2, 5]) == (2, 1)
assert two_pair([1, 1, 1, 2, 5]) == None
assert two_pair([10, 10, 5, 5, 2]) == (10, 5)

My version does not require `ranks` to be ordered

Peter:

In [17]:
def two_pair(ranks):
    """If there are two pair, return the two ranks as a
    tuple: (highest, lowest); otherwise return None."""

    ranks = sorted(ranks, reverse=True)

    pair = kind(2, ranks)
    lowpair = kind(2, list(reversed(ranks)))

    if pair and lowpair != pair:
        return (pair, lowpair)

    return None

In [18]:
assert two_pair([1, 2, 1, 2, 5]) == (2, 1)
assert two_pair([1, 1, 1, 2, 5]) == None
assert two_pair([10, 10, 5, 5, 2]) == (10, 5)

If there's a straight hand starting with ACE, the ace counts as 1, not 14.

How to handle it?

The amount of change in code should be proportional to the amount of change in the conceptualization of it.

1 change in concept ~= 1 change in code

In [19]:
def card_ranks(hand):
    "Return a list of the ranks, sorted with higher first."
    ranks = (
        ["--23456789TJQKA".index(r) for r, s in hand]
        if hand[0][0] != "A"
        else ["-A23456789TJQK".index(r) for r, s in hand]
    )
    ranks.sort(reverse=True)
    return ranks

Peter

In [20]:
def card_ranks(hand):
    "Return a list of the ranks, sorted with higher first."
    ranks = ["--23456789TJQKA".index(r) for r, s in hand]
    ranks.sort(reverse=True)
    return [5,4,3,2,1] if (ranks == [14,5,4,3,2]) else ranks

Handle ties

In [21]:
def allmax(iterable, key=None):
    "Return a list of all items equal to the max of the iterable."
    one_max = hand_rank(max(iterable, key=key))
    
    return [e for e in iterable if hand_rank(e) == one_max]

In [22]:
def poker(hands):
    "Return the best hand: poker([hand,...]) => hand"
    r = allmax(hands, key=hand_rank)
    print(r)
    return r

In [23]:
"Test cases for the functions in poker program."
sf1 = "6C 7C 8C 9C TC".split()  # Straight Flush
sf2 = "6D 7D 8D 9D TD".split()  # Straight Flush
fk = "9D 9H 9S 9C 7D".split()  # Four of a Kind
fh = "TD TC TH 7C 7D".split()  # Full House
assert poker([sf1, sf2, fk, fh]) == [sf1, sf2]

[['6C', '7C', '8C', '9C', 'TC'], ['6D', '7D', '8D', '9D', 'TD']]


In [24]:
allmax([sf1, sf2, fk, fh], key=hand_rank)

[['6C', '7C', '8C', '9C', 'TC'], ['6D', '7D', '8D', '9D', 'TD']]

Peter

(I like mine more)

In [25]:
def allmax(iterable, key=None):
    "Return a list of all items equal to the max of the iterable."
    result, maxval = [], None
    
    key = key or (lambda x: x)
    
    for x in iterable:
        # key will be hand_rank
        xval = key(x)
        
        if not result or xval > maxval:
            result, maxval = [x], xval
        elif xval == maxval:
            result.append(x)
            
    return result

Write function to deal cards

In [26]:
import random

In [27]:
mydeck = [r+s for r in '23456789TJQKA' for s in 'SHDC']

In [204]:
def deal(numhands, n=5, deck=mydeck):
    return [[random.choice(deck) for _ in range(n)] for _ in range(numhands)]

In [205]:
deal(5)

[['TH', '7H', 'TD', 'AC', '4D'],
 ['TD', 'TC', '3H', '6D', '5S'],
 ['QC', '4H', '3H', '6H', 'QD'],
 ['8S', '9C', '3C', '4C', '4H'],
 ['9H', 'KH', '9D', '3H', '9H']]

I noticed my implementation is wrong because the same card can be dealt multiple times (using `random.choice`.

As a fix, I could write it like:

In [78]:
def deal(numhands, n=5, fresh_deck=mydeck):

    # copy to avoid mutating original list (in python it's a reference)
    deck = fresh_deck.copy()
    # deck = copy.deepcopy(fresh_deck)
    # deck = [r+s for r in '23456789TJQKA' for s in 'SHDC']

    hands = []

    for i in range(numhands):
        hand = [random.choice(deck) for _ in range(n)]
        for card in hand:
            deck.remove(card)

        hands.append(hand)

    return hands


deal(5)

[['8S', '7H', '2C', '9S', '2H'],
 ['JC', 'QD', 'QH', '2D', 'TH'],
 ['TC', 'AD', '2S', 'JD', '9D'],
 ['7D', 'QC', '3H', '9C', '5C'],
 ['6D', '5D', 'TD', '5S', '4D']]

This implementation has a bug!! The function `random.choice` could select the same cart twice, then when doing the `remove` in the list, the second time if would throw an error.

In order to fix it, I need to use `random.sample()`

In [79]:
random.sample?

Signature: random.sample(population, k)
Docstring:
Chooses k unique random elements from a population sequence or set.

Returns a new list containing elements from the population while
leaving the original population unchanged.  The resulting list is
in selection order so that all sub-slices will also be valid random
samples.  This allows raffle winners (the sample) to be partitioned
into grand prize and second place winners (the subslices).

Members of the population need not be hashable or unique.  If the
population contains repeats, then each occurrence is a possible
selection in the sample.

To choose a sample in a range of integers, use range as an argument.
This is especially fast and space efficient for sampling from a
large population:   sample(range(10000000), 60)
File:      ~/.pyenv/versions/3.8.3/lib/python3.8/random.py
Type:      method


In [93]:
def deal(numhands, n=5, fresh_deck=mydeck):

    # copy to avoid mutating original list (in python it's a reference)
    deck = fresh_deck.copy()

    hands = []

    for i in range(numhands):
        hand = random.sample(deck, n)
        for card in hand:
            deck.remove(card)

        hands.append(hand)

    return hands


deal(5)

[['9C', 'JC', '3H', '5H', '4S'],
 ['9H', '3D', '7D', '3C', 'TD'],
 ['9S', 'KC', 'AH', '6S', 'KH'],
 ['AD', 'KD', 'AS', '7C', 'JD'],
 ['2H', '3S', '4H', '2S', 'TC']]

Alternative

In [96]:
def deal(numhands, n=5, fresh_deck=mydeck):

    # copy to avoid mutating original list (in python it's a reference)
    deck = fresh_deck.copy()

    cards = random.sample(deck, n * numhands)

    return [cards[n * i : n * (i + 1)] for i in range(numhands)]


deal(5)

[['4D', '9S', '8C', '5S', '3C'],
 ['2H', 'AD', 'TC', '5C', 'QH'],
 ['TH', '2D', 'JD', 'AH', 'JC'],
 ['4H', '9C', '5H', 'AS', 'KS'],
 ['JS', '3H', 'KD', 'QC', '6S']]

Peter

In [208]:
def deal(numhands, n=5, deck=mydeck):
    random.shuffle(deck)
    return [deck[n * i : n * (i + 1)] for i in range(numhands)]

In [207]:
deal(5)

[['AD', '5C', 'JD', '3H', 'QH'],
 ['6D', '3S', '4H', '7D', 'JC'],
 ['AC', 'JS', '6H', 'TD', '2D'],
 ['AS', 'QD', '4C', '5H', '9S'],
 ['TH', '8C', '8S', '5S', '3C']]

**The 4 dimensions of programming**

* Efficiency
* Feature
* Elegance (clarity, simplicity, generality)  <== refactorings.
* Correctnes


*The best is the enemy of the good* - Voltaire

Do it good, but don't obesses over "best". You may lose the time. Learn to make good tradeofs.

**Solving a problem**

* Understand the problem: check the specification, does it make sense?, discuss it...
* Define pieces of the problem
* Re-use the pieces
* Test
* Explore: decide where you want to be in the 4 dimensions of programming (mentioned above)

Homework

Find best 5-card hand from 7 cards.

In [134]:
import itertools


def best_hand(hand):
    "From a 7-card hand, return the best 5 card hand."

    curr_max = (0, 0, 0)
    best_hand = None

    for comb in itertools.combinations(hand, 5):
#         hand = [(e[0], e[-1]) for e in comb]
        if hand_rank(comb) > curr_max:
            curr_max = hand_rank(comb)
            best_hand = comb

    return list(best_hand)

Peter

In [136]:
def best_hand(hand):
    return max(itertools.combinations(hand, 5), key=hand_rank)

In [135]:
# ------------------
# Provided Functions
#
# You may want to use some of the functions which
# you have already defined in the unit to write
# your best_hand function.


def hand_rank(hand):
    "Return a value indicating the ranking of a hand."
    ranks = card_ranks(hand)
    if straight(ranks) and flush(hand):
        return (8, max(ranks))
    elif kind(4, ranks):
        return (7, kind(4, ranks), kind(1, ranks))
    elif kind(3, ranks) and kind(2, ranks):
        return (6, kind(3, ranks), kind(2, ranks))
    elif flush(hand):
        return (5, ranks)
    elif straight(ranks):
        return (4, max(ranks))
    elif kind(3, ranks):
        return (3, kind(3, ranks), ranks)
    elif two_pair(ranks):
        return (2, two_pair(ranks), ranks)
    elif kind(2, ranks):
        return (1, kind(2, ranks), ranks)
    else:
        return (0, ranks)


def card_ranks(hand):
    "Return a list of the ranks, sorted with higher first."
    ranks = ["--23456789TJQKA".index(r) for r, s in hand]
    ranks.sort(reverse=True)
    return [5, 4, 3, 2, 1] if (ranks == [14, 5, 4, 3, 2]) else ranks


def flush(hand):
    "Return True if all the cards have the same suit."
    suits = [s for r, s in hand]
    return len(set(suits)) == 1


def straight(ranks):
    """Return True if the ordered
    ranks form a 5-card straight."""
    return (max(ranks) - min(ranks) == 4) and len(set(ranks)) == 5


def kind(n, ranks):
    """Return the first rank that this hand has
    exactly n-of-a-kind of. Return None if there
    is no n-of-a-kind in the hand."""
    for r in ranks:
        if ranks.count(r) == n:
            return r
    return None


def two_pair(ranks):
    """If there are two pair here, return the two
    ranks of the two pairs, else None."""
    pair = kind(2, ranks)
    lowpair = kind(2, list(reversed(ranks)))
    if pair and lowpair != pair:
        return (pair, lowpair)
    else:
        return None


def test_best_hand():
    assert sorted(best_hand("6C 7C 8C 9C TC 5C JS".split())) == [
        "6C",
        "7C",
        "8C",
        "9C",
        "TC",
    ]
    assert sorted(best_hand("TD TC TH 7C 7D 8C 8S".split())) == [
        "8C",
        "8S",
        "TC",
        "TD",
        "TH",
    ]
    assert sorted(best_hand("JD TC TH 7C 7D 7S 7H".split())) == [
        "7C",
        "7D",
        "7H",
        "7S",
        "JD",
    ]
    return "test_best_hand passes"


test_best_hand()

'test_best_hand passes'

In [137]:
# Write a function best_wild_hand(hand) that takes as
# input a 7-card hand and returns the best 5 card hand.
# In this problem, it is possible for a hand to include
# jokers. Jokers will be treated as 'wild cards' which
# can take any rank or suit of the same color. The
# black joker, '?B', can be used as any spade or club
# and the red joker, '?R', can be used as any heart
# or diamond.
#
# The itertools library may be helpful. Feel free to
# define multiple functions if it helps you solve the
# problem.
#
# -----------------
# Grading Notes
#
# Muliple correct answers will be accepted in cases
# where the best hand is ambiguous (for example, if
# you have 4 kings and 3 queens, there are three best
# hands: 4 kings along with any of the three queens).

In [141]:
blues = [r + s for r in "23456789TJQKA" for s in "SC"]
reds = [r + s for r in "23456789TJQKA" for s in "HD"]

In [ ]:
def best_wild_hand(hand):
    "Try all values for jokers in all 5-card selections."


assert sorted(best_wild_hand("6C 7C 8C 9C TC 5C ?B".split())) == [
    "7C",
    "8C",
    "9C",
    "JC",
    "TC",
]
assert sorted(best_wild_hand("TD TC 5H 5C 7C ?R ?B".split())) == [
    "7C",
    "TC",
    "TD",
    "TH",
    "TS",
]
assert sorted(best_wild_hand("JD TC TH 7C 7D 7S 7H".split())) == [
    "7C",
    "7D",
    "7H",
    "7S",
    "JD",
]

In [103]:
def f(x):
    return x + 1

In [104]:
def ff(x):
    return x + 2

In [105]:
m = {1: 2, 3: 4, "asd": [("aa", 2), (f, 4)], ff: 1}

In [108]:
m[ff]

1